In [ ]:
import numpy as np
import matplotlib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import util


%matplotlib inline

In [ ]:
dataset = np.load('./datasets/v0_eigens.npz')

train_data_size = dataset['train_eigens'].shape[0]
valid_data_size = train_data_size / 5
train_data_size = train_data_size - valid_data_size

indices = np.arange(train_data_size + valid_data_size)

# np.random.shuffle(indices)

train_data = dataset['train_eigens'][indices[:train_data_size]]
valid_data = dataset['train_eigens'][indices[train_data_size:]]

train_eigens = train_data[:, :-28]
train_labels = train_data[:, -28:]
valid_eigens = valid_data[:, :-28]
valid_labels = valid_data[:, -28:]

print 'train_eigens.shape = {}'.format(train_eigens.shape)
print 'train_labels.shape = {}'.format(train_labels.shape)
print 'valid_eigens.shape = {}'.format(valid_eigens.shape)
print 'valid_labels.shape = {}'.format(valid_labels.shape)

In [ ]:
batch_size = 256
learning_rate = 0.00001

w = (np.random.rand(train_eigens.shape[1], 28) * 2.0 - 1.0) * 0.01
b = np.zeros((1, 28))

valid_guesss = np.dot(valid_eigens, w) + b

auc = util.auc(valid_guesss.flatten(), valid_labels.flatten())

print 'auc[before]: {}'.format(auc)

for step, eigens, labels in util.mini_batches(train_eigens, train_labels, batch_size, False):
    y = np.dot(eigens, w) + b
    
    loss = np.sum(np.square(y - labels), axis=1)

    if step % 5000 == 0:
        print 'loss[{:>8}]: {}'.format(step, np.mean(loss))
    
    dl_dy = 2.0 * (y - labels)
    dl_dw = np.mean(np.dot(eigens.T, dl_dy), axis=0)
    dl_db = np.mean(dl_dy, axis=0)
    
    w = w - learning_rate * dl_dw
    b = b - learning_rate * dl_db
    
    if step == 50000:
        break
        
valid_guesss = np.dot(valid_eigens, w) + b

auc = util.auc(valid_guesss.flatten(), valid_labels.flatten())

print 'auc[after]: {}'.format(auc)

In [ ]:
image_inception = mpimg.imread('./datasets/inception.png')

plt.imshow(image_inception)

In [ ]:
batch_size = 256
learning_rate = 0.000008

w1 = (np.random.rand(train_eigens.shape[1], 128) * 2.0 - 1.0) * 0.01
b1 = np.zeros((1, 128))
w2 = (np.random.rand(128, 28) * 2.0 - 1.0) * 0.01
b2 = np.zeros((1, 28))

y1 = np.dot(valid_eigens, w1) + b1
y1[y1 < 0.0] = 0.0

y2 = np.dot(y1, w2) + b2
y2 = 1.0 / (1.0 + np.exp(-y2))

valid_guesss = y2

auc = util.auc(valid_guesss.flatten(), valid_labels.flatten())

print 'auc[before]: {}'.format(auc)

for step, eigens, labels in util.mini_batches(train_eigens, train_labels, batch_size, False):
    y1 = np.dot(eigens, w1) + b1
    y1[y1 < 0.0] = 0.0

    y2 = np.dot(y1, w2) + b2
   
    loss = np.sum(np.square(y2 - labels), axis=1)

    if step % 5000 == 0:
        print 'loss[{:>8}]: {}'.format(step, np.mean(loss))
    
    dl_dy2 = 2.0 * (y2 - labels)
    dl_dw2 = np.mean(np.dot(y1.T, dl_dy2), axis=0)
    dl_db2 = np.mean(dl_dy2, axis=0)

    dl_dy1 = np.dot(dl_dy2, w2.T)
    dl_dw1 = np.mean(np.dot(eigens.T, dl_dy1), axis=0)
    dl_db1 = np.mean(dl_dy1, axis=0)
    
    w2 = w2 - learning_rate * dl_dw2
    b2 = b2 - learning_rate * dl_db2
    
    w1 = w1 - learning_rate * dl_dw1
    b1 = b1 - learning_rate * dl_db1
    
    if step == 50000:
        break
        
y1 = np.dot(valid_eigens, w1) + b1
y1[y1 < 0.0] = 0.0

y2 = np.dot(y1, w2) + b2
y2 = 1.0 / (1.0 + np.exp(-y2))

valid_guesss = y2

auc = util.auc(valid_guesss.flatten(), valid_labels.flatten())

print 'auc[after]: {}'.format(auc)